In [28]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.config("spark.sql.shuffle.partitions", "2").appName("Analysis").master("local[2]").getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [29]:
%run "../includes/configuration"
%run "../includes/common_functions"

In [30]:
circuits_df = spark.read.parquet(f"{processed_folder_path}/circuits").withColumnRenamed("name", "circuit_name")

In [31]:
races_df = spark.read.parquet(f"{processed_folder_path}/races").filter("race_year = 2019").withColumnRenamed("name", "race_name")

In [32]:
races_df.show(5)

+-------+-----+----------+--------------------+--------------------+-------------------+---------+
|race_id|round|circuit_id|           race_name|      ingestion_date|     race_timestamp|race_year|
+-------+-----+----------+--------------------+--------------------+-------------------+---------+
|   1010|    1|         1|Australian Grand ...|2023-08-16 18:28:...|2019-03-17 05:10:00|     2019|
|   1011|    2|         3|  Bahrain Grand Prix|2023-08-16 18:28:...|2019-03-31 15:10:00|     2019|
|   1012|    3|        17|  Chinese Grand Prix|2023-08-16 18:28:...|2019-04-14 06:10:00|     2019|
|   1013|    4|        73|Azerbaijan Grand ...|2023-08-16 18:28:...|2019-04-28 12:10:00|     2019|
|   1014|    5|         4|  Spanish Grand Prix|2023-08-16 18:28:...|2019-05-12 13:10:00|     2019|
+-------+-----+----------+--------------------+--------------------+-------------------+---------+
only showing top 5 rows



In [33]:
race_circuits_df = circuits_df.join(races_df, circuits_df.circuit_id == races_df.circuit_id, "inner")\
.select(circuits_df.circuit_name, circuits_df.location, circuits_df.country, races_df.race_name, races_df.round)

In [34]:
race_circuits_df.show()

+--------------------+------------+----------+--------------------+-----+
|        circuit_name|    location|   country|           race_name|round|
+--------------------+------------+----------+--------------------+-----+
|Albert Park Grand...|   Melbourne| Australia|Australian Grand ...|    1|
|Bahrain Internati...|      Sakhir|   Bahrain|  Bahrain Grand Prix|    2|
|Shanghai Internat...|    Shanghai|     China|  Chinese Grand Prix|    3|
|   Baku City Circuit|        Baku|Azerbaijan|Azerbaijan Grand ...|    4|
|Circuit de Barcel...|    Montmeló|     Spain|  Spanish Grand Prix|    5|
|   Circuit de Monaco| Monte-Carlo|    Monaco|   Monaco Grand Prix|    6|
|Circuit Gilles Vi...|    Montreal|    Canada| Canadian Grand Prix|    7|
| Circuit Paul Ricard|Le Castellet|    France|   French Grand Prix|    8|
|       Red Bull Ring|   Spielburg|   Austria| Austrian Grand Prix|    9|
| Silverstone Circuit| Silverstone|        UK|  British Grand Prix|   10|
|      Hockenheimring|  Hockenheim|   